In [1]:
import pandas as pd
import numpy as np
import cv2
import IPython.display as ipd
import librosa
import ndjson
import os

# Video dataframe

In [2]:
ipd.Video("D:\\machine project\\data\\videos\\cmhphcl8x0m3l0766dfoqpmje.mp4", width=700, embed=True)


In [3]:

# Open and read the NDJSON file
with open('video-input\\videos.ndjson', 'r') as f:
    data = ndjson.load(f)


print(f"Combined records: {len(data)}")


Combined records: 69


In [4]:
df_meta = pd.DataFrame(data)
df_meta.drop(columns=['media_attributes'], inplace=True)
df_meta['data_row'] = df_meta['data_row'].astype(str).str[8:37]
df_meta.rename(columns={'data_row': 'id'}, inplace=True)
df_meta.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta.head()

,id,emotion
0,cmhpke9p4cjbs0782sob057xx.mp4,[{'value': 'Surprised'}]
1,cmhpke9p4cjbt0782363n2x0u.mp4,[{'value': 'Surprised'}]
2,cmhpke9p4cjbu0782assgc5x9.mp4,[{'value': 'Surprised'}]
3,cmhpke9p4cjbv0782ah5bfrg4.mp4,[{'value': 'Surprised'}]
4,cmhpke9p4cjbw0782wkp7yx3s.mp4,[{'value': 'Surprised'}]


In [5]:
from skimage.feature import hog, local_binary_pattern

casc_path = (r"C:\\Users\\PC\\anaconda3\\pkgs\\opencv-4.12.0-py312h6fcf9f4_0\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml")

class EmotionFeatureExtractor:
    def __init__(self):
        # Load Haar Cascade
        self.face_cascade = cv2.CascadeClassifier(casc_path)
        
        # INPUT SIZE
        self.resize_dim = (64, 64) 
        
        # LBP SETTINGS
        self.lbp_radius = 1
        self.lbp_points = 8 * self.lbp_radius

    def get_features(self, frame):
        # 1. PREPROCESSING
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        
        # 2. FACE DETECTION
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
        
        # If no face is found, Return None.
        if len(faces) == 0:
            return None
        
        # Since there is only one face, we just take the first one in the list.
        (x, y, w, h) = faces[0]
        
        # Crop the face
        face_roi = gray[y:y+h, x:x+w]
        
        # CHECK 2: Is the cropped face valid? (Prevents crash if face is off-screen)
        if face_roi.shape[0] == 0 or face_roi.shape[1] == 0:
            return None
        
        # Resize to 64x64 so every feature vector is the exact same length
        face_roi = cv2.resize(face_roi, self.resize_dim)
        
        # 4. HOG FEATURES (Shape)
        # Returns a 1D array (vector)
        hog_feats = hog(face_roi, orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), visualize=False)
        
        # 5. LBP FEATURES (Texture)
        # Generate the LBP pattern image
        lbp_image = local_binary_pattern(face_roi, self.lbp_points, self.lbp_radius, method="uniform")
        
        # Convert 2D image to 1D Histogram (counts of patterns)
        (lbp_hist, _) = np.histogram(lbp_image.ravel(), 
                                     bins=np.arange(0, self.lbp_points + 3), 
                                     range=(0, self.lbp_points + 2))
        
        # Normalize LBP so it doesn't overpower HOG (Turn counts into probabilities)
        lbp_hist = lbp_hist.astype("float")
        lbp_hist /= (lbp_hist.sum() + 1e-7)

        # 6. COMBINE
        final_vector = np.concatenate([hog_feats, lbp_hist])
        
        return final_vector

In [6]:
def process_video_dataset(skip_frames):
    
    data_rows = []
    extractor = EmotionFeatureExtractor()
    
    ROOT_DIR = r"D:\machine project\data\videos"

    for filename in os.listdir(ROOT_DIR):
        if filename.endswith(('.mp4')):
            
            # CRUCIAL: os.path.join handles backslashes correctly for the OS!
            video_path = os.path.join(ROOT_DIR, filename)
            
            # Get the ID (filename including extension)
            video_id = os.path.basename(video_path) 
            
            print(f"Processing: {video_id}")
            
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            
            while True:
                ret, frame = cap.read()
                if not ret: break
                
                if frame_count % skip_frames == 0:

                    # Create flipped version for augmentation
                    flipped_frame = cv2.flip(frame, 1)
                    
                    # --- EXTRACT FEATURES ---
                    features = extractor.get_features(frame)
                    features_flip = extractor.get_features(flipped_frame)
                    # --- AUGMENTATION & APPEND ---
                    
                    # 1. Original Frame
                    if features is not None and features_flip is not None:
                        row = [video_id] + features.tolist()
                        data_rows.append(row)
                        
                        # 2. Flipped Frame (Augmentation)
                        row_flip = [video_id] + features_flip.tolist()
                        data_rows.append(row_flip)
                    else:
                        print(f"  Warning: No face detected in frame {frame_count} of video {video_id}. Skipping frame.")
                        pass


                frame_count += 1
            
            cap.release()

    return data_rows

In [7]:

# Define a filename for your "saved brain"
CACHE_FILE = "video_features_data.pkl"

# 1. CHECK IF SAVE FILE EXISTS
if os.path.exists(CACHE_FILE):
    print(f"🚀 Found '{CACHE_FILE}'! Loading data directly from disk...")
    print("   (This should take only a few seconds)")
    
    # Load the dataframe instantly
    df_final = pd.read_pickle(CACHE_FILE)

else:
    print(f"🐢 No cache found. Starting heavy video processing...")
    
    # 2. RUN THE HEAVY PROCESSING (Only happens once!)
    # Call your function
    DATA_ROWS = process_video_dataset(skip_frames=2) 
    
    # Create DataFrame
    num_features = len(DATA_ROWS[0]) - 1
    column_names = ['id'] + [f'pixel_{i}' for i in range(num_features)]
    df_features = pd.DataFrame(DATA_ROWS, columns=column_names)
    
    # Merge with metadata (assuming df_meta is loaded)
    df_final = pd.merge(df_features, df_meta, on='id', how='left')
    
    # 3. SAVE TO PICKLE
    print("💾 Saving processed data to disk for next time...")
    df_final.to_pickle(CACHE_FILE)

print(f"✅ Data Ready. Shape: {df_final.shape}")

🚀 Found 'video_features_data.pkl'! Loading data directly from disk...
   (This should take only a few seconds)
✅ Data Ready. Shape: (3740, 524)


In [8]:
# DATA_ROWS = process_video_dataset(skip_frames=2)

# if len(DATA_ROWS) > 0:
#     # Determine the number of feature columns (Total length - 2 for 'id')
#     num_features = len(DATA_ROWS[0]) - 1
    
#     # Create column names: ['id', '0', '1', ... '4095']
#     column_names = ['id'] + [f'pixel_{i}' for i in range(num_features)]
    
#     df_features = pd.DataFrame(DATA_ROWS, columns=column_names)
    
#     print("\n--- Final DataFrame Status ---")
#     print(f"Total rows (frames) extracted: {df_features.shape[0]}")
#     print(f"Total features per row: {df_features.shape[1]}") # Should be 4098
#     print(df_features[['id', 'pixel_0', 'pixel_1']].head())

In [9]:
#df_features

In [10]:
# df_final = pd.merge(
#     df_features, 
#     df_meta,
#     on='id', 
#     how='left'
# )

# print(f"Original Feature Shape: {df_features.shape}")
# print(f"Final Merged Shape:     {df_final.shape}")
# print("\nFirst 5 rows of the merged data, showing the copied emotion:")
# print(df_final[['id', 'emotion', 'pixel_0']].head())
# df_final['emotion'].value_counts()

# audio dataframe

In [11]:
ipd.Audio('data\\audio\\cmi6kb42m14t80768b3k66mu7.WAV')

In [12]:
y, sr = librosa.load('data\\audio\\cmi6kb42m14t80768b3k66mu7.WAV')
print(f'y: {y[:10]}')
print(f'shape y: {y.shape}')
print(f'sr: {sr}')

y: [-0.0041386  -0.00724762 -0.00685604 -0.00601203 -0.00392585 -0.00262255
 -0.00035936  0.00160185  0.00338469  0.0046488 ]
shape y: (48128,)
sr: 22050


In [13]:
y_trimmed, index = librosa.effects.trim(y, top_db=20)
ipd.Audio(y_trimmed, rate=sr)

In [14]:


# Open and read the NDJSON file
with open('video-input\\Audio.ndjson', 'r') as f:
    data = ndjson.load(f)


print(f"Combined records: {len(data)}")



Combined records: 50


In [15]:
df_audio_meta = pd.DataFrame(data)
df_audio_meta.drop(columns=['media_attributes'], inplace=True)
df_audio_meta['data_row'] = df_audio_meta['data_row'].astype(str).str[8:37]
df_audio_meta.rename(columns={'data_row': 'id'}, inplace=True)
df_audio_meta.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_audio_meta.head()

,id,emotion
0,cmi6kb42m14t80768b3k66mu7.WAV,[{'value': 'Angry'}]
1,cmi6kb42m14t907681xw39xws.WAV,[{'value': 'Angry'}]
2,cmi6kb42m14ta0768107er0vz.WAV,[{'value': 'Angry'}]
3,cmi6kb42m14tb0768roy88bhh.WAV,[{'value': 'Angry'}]
4,cmi6kb42m14tc076895ghj9du.WAV,[{'value': 'Angry'}]


In [16]:


class AudioFeatureExtractor:
    def __init__(self):
        self.n_mfcc = 40
        self.segment_duration = 1  # Chunk size in seconds

    def get_features(self, file_path):
        try:
            # 1. LOAD FULL AUDIO
            y, sr = librosa.load(file_path, sr=22050)
            
            # Trim silence from the very start and end of the file
            y_trimmed, _ = librosa.effects.trim(y, top_db=20)
            
            # Calculate how many samples represent 0.5 seconds
            # (22050 samples/sec * 0.5 sec = 11025 samples)
            samples_per_chunk = int(self.segment_duration * sr)
            total_samples = len(y_trimmed)
            
            all_chunks_features = []

            # 2. LOOP THROUGH THE AUDIO IN 0.5s STEPS
            for start_idx in range(0, total_samples, samples_per_chunk):
                end_idx = start_idx + samples_per_chunk
                
                # Check: Is this chunk full size?
                # If the last piece is tiny (e.g., 0.1s), we ignore it to avoid bad data
                if end_idx > total_samples:
                    break
                
                # Slice the audio: Get just this 0.5s piece
                y_chunk = y_trimmed[start_idx : end_idx]
                
                # --- EXTRACT FEATURES FOR THIS CHUNK ---
                if len(y_chunk) > 0:
                    mfcc = librosa.feature.mfcc(y=y_chunk, sr=sr, n_mfcc=self.n_mfcc)
                    mfcc_mean = np.mean(mfcc, axis=1)
                    mfcc_std = np.std(mfcc, axis=1)
                    
                    # Combine into vector (Size 80)
                    chunk_vector = np.concatenate([mfcc_mean, mfcc_std])
                    all_chunks_features.append(chunk_vector)
            
            # Return a LIST of vectors (one for each 0.5s moment)
            return all_chunks_features

        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            return []

In [17]:
def process_audio_dataset():
    data_rows = []
    extractor = AudioFeatureExtractor()
    
    ROOT_DIR = r"D:\machine project\data\audio"

    print("✂️  Slicing audio files into segments...")

    # Loop through all files directly inside the ROOT_DIR
    for filename in os.listdir(ROOT_DIR):
        if filename.lower().endswith(('.wav', '.mp4', '.mp3')):
            
            video_path = os.path.join(ROOT_DIR, filename)
            # We keep the original ID (e.g. "video1.wav") so we can merge with emotions later
            original_id = os.path.basename(video_path) 
            
            # --- EXTRACT AUDIO FEATURES ---
            # Now returns a LIST of vectors (one for each 0.5s chunk)
            chunk_list = extractor.get_features(video_path)
            
            # Check if list is not empty
            if chunk_list:
                # NEW LOOP: Iterate through every 0.5s chunk
                for i, vector in enumerate(chunk_list):
                    
                    # vector is a NumPy array, so .tolist() works here!
                    # We add the original_id so this chunk gets the right label later
                    row = [original_id] + vector.tolist()
                    data_rows.append(row)

    print(f"✅ Processing Complete. Generated {len(data_rows)} total segments.")
    return data_rows


In [18]:

# --- RUN AND BUILD DATAFRAME ---
AUDIO_DATA_ROWS = process_audio_dataset()

if len(AUDIO_DATA_ROWS) > 0:
    # Calculate feature columns (Total length - 1 for 'id')
    num_features = len(AUDIO_DATA_ROWS[0]) - 1 
    
    column_names = ['id'] + [f'mfcc_{i}' for i in range(num_features)]
    
    df_audio_features = pd.DataFrame(AUDIO_DATA_ROWS, columns=column_names)
    
    print(f"\nExample of Segmented Data:")
    print(df_audio_features.head())
    print(f"Total Rows (Segments): {len(df_audio_features)}")
    

✂️  Slicing audio files into segments...
✅ Processing Complete. Generated 83 total segments.

Example of Segmented Data:
                              id      mfcc_0      mfcc_1     mfcc_2  \
0  cmi6k8w160dci0751dqq8vv7f.WAV -392.816528  153.789062 -30.854057   
1  cmi6k8w160dci0751dqq8vv7f.WAV -348.582794  134.971207   0.336861   
2  cmi6k8w160dcj075151x2h6h2.WAV -341.228149  136.565826 -13.133108   
3  cmi6k8w160dck0751xfihqptv.WAV -211.363678  110.305496 -18.134178   
4  cmi6k8w160dcm0751pg64lonv.WAV -269.820038  110.553886   6.746582   

      mfcc_3     mfcc_4     mfcc_5     mfcc_6     mfcc_7    mfcc_8  ...  \
0  45.182072 -10.605449  26.844086   3.368268  11.776779 -7.383310  ...   
1  28.593651  -6.962745  13.999433   4.647218   4.519924 -6.972414  ...   
2  41.254898  -0.648194   1.704550   7.135161  -2.582593  3.249355  ...   
3  21.857737 -21.149654  -1.412924  18.668537 -25.351290  7.446501  ...   
4  27.547884  -4.314253  -0.669958   3.174553  -2.005161  6.989856  ...   

 

In [19]:
df_Audio = pd.merge(
    df_audio_features, 
    df_audio_meta,  # Only select the columns you need from df_audio_meta
    on='id', 
    how='left'
)

print(f"Original Feature Shape: {df_audio_features.shape}")
print(f"Final Merged Shape:     {df_Audio.shape}")
print("\nFirst 5 rows of the merged data, showing the copied emotion:")
print(df_Audio[['id', 'mfcc_1', 'emotion']])

Original Feature Shape: (83, 81)
Final Merged Shape:     (83, 82)

First 5 rows of the merged data, showing the copied emotion:
                               id      mfcc_1                   emotion
0   cmi6k8w160dci0751dqq8vv7f.WAV  153.789062  [{'value': 'Surprised'}]
1   cmi6k8w160dci0751dqq8vv7f.WAV  134.971207  [{'value': 'Surprised'}]
2   cmi6k8w160dcj075151x2h6h2.WAV  136.565826  [{'value': 'Surprised'}]
3   cmi6k8w160dck0751xfihqptv.WAV  110.305496  [{'value': 'Surprised'}]
4   cmi6k8w160dcm0751pg64lonv.WAV  110.553886  [{'value': 'Surprised'}]
..                            ...         ...                       ...
78  cmi6kdbtk2t3l0743zltxt31w.WAV  138.423462      [{'value': 'Happy'}]
79  cmi6kdbtk2t3n0743o9m97lw9.WAV  151.011902      [{'value': 'Happy'}]
80  cmi6kdbtk2t3n0743o9m97lw9.WAV  138.223007      [{'value': 'Happy'}]
81  cmi6kdbtk2t3n0743o9m97lw9.WAV  145.834702      [{'value': 'Happy'}]
82  cmi6kdbtk2t3o0743g5jmbaye.WAV  129.291061      [{'value': 'Happy'}]

[83 row

In [ ]:
# --- ROBUST DATA CLEANING ---
def clean_emotion_col(x):
    # 1. If it's a list (e.g., ['angry']), grab the first item
    if isinstance(x, list):
        if len(x) > 0:
            x = x[0]
        else:
            return "unknown"

    # 2. If it's a dictionary (e.g., {'label': 'angry'}), grab the value
    if isinstance(x, dict):
        # Try common keys, or just grab the first value found
        x = x.get('label', x.get('emotion', list(x.values())[0]))

    # 3. Convert whatever is left to a string and lower-case it
    text = str(x).lower()
    
    # 4. Remove any leftover garbage characters
    text = text.replace("'", "").replace('"', "").strip()
    return text

# Apply the fix
print("Cleaning data...")
df_final['emotion'] = df_final['emotion'].apply(clean_emotion_col)
df_Audio['emotion'] = df_Audio['emotion'].apply(clean_emotion_col)
# --- VERIFY ---
print("\nUnique emotions found:")
print(df_final['emotion'].unique())
print(df_Audio['emotion'].unique())

Cleaning data...

Unique emotions found:
['happy' 'sad' 'neutral' 'angry' 'disgust' 'surprised']
['surprised' 'disgust' 'angry' 'neutral' 'sad' 'happy']


# SVM

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# --- IMPORT THE 5 MODELS ---
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# ==========================================
# CONFIGURATION
# ==========================================
MODELS = {
    'svm': SVC(kernel='rbf', C=10.0, probability=True, class_weight='balanced', random_state=42),
    'rf': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'gb': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'lr': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
}

# ==========================================
# 1. TRAIN AUDIO MODELS (3-Class: Angry, Happy, Other)
# ==========================================
def train_all_audio_models(df):
    print("\n🎧 --- STARTING AUDIO TRAINING (3 Classes) ---")
    
    # 1. Prepare Data
    feature_cols = [col for col in df.columns if col.startswith('mfcc_')]
    X = df[feature_cols].values
    y_original = df['emotion'].values

    # Group Labels
    y_grouped = []
    for label in y_original:
        l = str(label).lower()
        if 'angry' in l: y_grouped.append('angry')
        elif 'happy' in l: y_grouped.append('happy')
        else: y_grouped.append('other')
    
    y = np.array(y_grouped)
    
    # Encode & Split
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)
    
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Save Common Tools (Scaler/Encoder are same for all models)
    joblib.dump(scaler, 'audio_scaler_common.pkl')
    joblib.dump(le, 'audio_label_encoder_common.pkl')

    # Train Loop
    for name, model in MODELS.items():
        print(f"   Training Audio {name.upper()}...", end=" ")
        model.fit(X_train_scaled, y_train)
        acc = accuracy_score(y_test, model.predict(X_test_scaled))
        print(f"Done. Accuracy: {acc*100:.1f}%")
        
        # Save Model
        joblib.dump(model, f'audio_model_{name}.pkl')

# ==========================================
# 2. TRAIN VIDEO MODELS (6-Class: Full Emotions)
# ==========================================
def train_all_video_models(df):
    print("\n👁️ --- STARTING VIDEO TRAINING (6 Classes) ---")
    
    # 1. Prepare Data
    feature_cols = [col for col in df.columns if col.startswith('pixel_')]
    X = df[feature_cols].values
    
    
    # Encode & Split
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)
    
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Save Common Tools
    joblib.dump(scaler, 'video_scaler_common.pkl')
    joblib.dump(le, 'video_label_encoder_common.pkl')

    # Train Loop
    for name, model in MODELS.items():
        print(f"   Training Video {name.upper()}...", end=" ")
        model.fit(X_train_scaled, y_train)
        acc = accuracy_score(y_test, model.predict(X_test_scaled))
        print(f"Done. Accuracy: {acc*100:.1f}%")
        
        # Save Model
        joblib.dump(model, f'video_model_{name}.pkl')

# --- EXECUTE ---
# Assuming df_Audio and df_final are already loaded in your environment!
if 'df_Audio' in locals() and 'df_final' in locals():
    train_all_audio_models(df_Audio)
    train_all_video_models(df_final)
    print("\n✅ ALL 10 MODELS TRAINED & SAVED SUCCESSFULLY.")
else:
    print("❌ Error: df_Audio or df_final not found. Please load your dataframes first.")


🎧 --- STARTING AUDIO TRAINING (3 Classes) ---
   Training Audio SVM... Done. Accuracy: 88.2%
   Training Audio RF... Done. Accuracy: 82.4%
   Training Audio KNN... Done. Accuracy: 70.6%
   Training Audio GB... Done. Accuracy: 88.2%
   Training Audio LR... Done. Accuracy: 94.1%

👁️ --- STARTING VIDEO TRAINING (6 Classes) ---
   Training Video SVM... Done. Accuracy: 95.9%
   Training Video RF... Done. Accuracy: 90.6%
   Training Video KNN... Done. Accuracy: 94.9%
   Training Video GB... Done. Accuracy: 89.7%
   Training Video LR... Done. Accuracy: 88.8%

✅ ALL 10 MODELS TRAINED & SAVED SUCCESSFULLY.


In [23]:


# --- 1. PREPARE THE DATA (FULL 6-CLASS MODE) ---
print("\n--- Phase 2: Training Video Classifier (All Classes) ---")



# B. DEFINE X AND Y (No Grouping!)
feature_cols = [col for col in df_final.columns if col.startswith('pixel_')]
X = df_final[feature_cols].values
y = df_final['emotion'].values

# Check what we are working with
unique_emotions, counts = np.unique(y, return_counts=True)
print(f"Emotions found: {dict(zip(unique_emotions, counts))}")

# --- 2. ENCODING & SPLITTING ---
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Save the full map (e.g., 0=Angry, 1=Disgust, 2=Fear...)
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(f"Label Mapping: {label_mapping}")

# STRATIFY is vital here to ensure rare classes (like 'fear') appear in the test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# --- 3. SCALING ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 4. TRAIN THE SVM ---
print("Training Video SVM Model... (This takes longer with 6 classes)")

# gamma='scale' is better for high-dimensional data (like 4000+ pixel features)
video_model = SVC(kernel='rbf', C=10.0, gamma='scale', probability=True, 
                  class_weight='balanced', random_state=42)

video_model.fit(X_train_scaled, y_train)

# --- 5. EVALUATE ---
print("\n--- Results ---")
y_pred = video_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Video Model Accuracy: {accuracy*100:.2f}%")

# Robust Reporting
unique_test_labels = np.unique(y_test)
target_names_test = le.inverse_transform(unique_test_labels)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names_test))

# --- 6. SAVE ---
if accuracy > 0.60:
    joblib.dump(video_model, 'video_emotion_model_full.pkl')
    joblib.dump(scaler, 'video_scaler_full.pkl')
    joblib.dump(le, 'video_label_encoder_full.pkl')
    print("💾 Files saved with suffix '_full'")
else:
    print("⚠️ Accuracy too low. Review the Classification Report.") 


--- Phase 2: Training Video Classifier (All Classes) ---
Emotions found: {'angry': np.int64(304), 'disgust': np.int64(328), 'happy': np.int64(1128), 'neutral': np.int64(876), 'sad': np.int64(734), 'surprised': np.int64(370)}
Label Mapping: {'angry': np.int64(0), 'disgust': np.int64(1), 'happy': np.int64(2), 'neutral': np.int64(3), 'sad': np.int64(4), 'surprised': np.int64(5)}
Training Video SVM Model... (This takes longer with 6 classes)

--- Results ---
✅ Video Model Accuracy: 95.86%

Classification Report:
              precision    recall  f1-score   support

       angry       1.00      0.90      0.95        61
     disgust       1.00      0.95      0.98        65
       happy       0.93      0.98      0.95       226
     neutral       0.94      0.97      0.95       175
         sad       0.99      0.97      0.98       147
   surprised       0.99      0.91      0.94        74

    accuracy                           0.96       748
   macro avg       0.97      0.95      0.96       7

# Fusion

In [29]:
class AngryMeter:
    def __init__(self):
        print("🔄 Initializing Fusion Engine...")
        
        # 1. Load SCALERS and ENCODERS (These are shared across all models)
        self.audio_scaler = joblib.load('audio_scaler_common.pkl')
        self.audio_le = joblib.load('audio_label_encoder_common.pkl')
        self.video_scaler = joblib.load('video_scaler_common.pkl')
        self.video_le = joblib.load('video_label_encoder_common.pkl')

        self.video_extractor = EmotionFeatureExtractor()

        # Config
        self.WEIGHT_VIDEO = 0.7
        self.WEIGHT_AUDIO = 0.3
        self.n_mfcc = 40
        self.audio_angry_idx = list(self.audio_le.classes_).index('angry')
        self.video_angry_idx = list(self.video_le.classes_).index('angry') if 'angry' in self.video_le.classes_ else 0
        
        # Load Default Model (SVM)
        self.current_model_name = "None"
        self.load_models("svm")
        
        print("✅ System Ready.")

    def load_models(self, model_name):
        """Swaps the brain of the AI on the fly"""
        try:
            print(f"🔄 Switching to {model_name.upper()}...")
            self.audio_model = joblib.load(f'audio_model_{model_name}.pkl')
            self.video_model = joblib.load(f'video_model_{model_name}.pkl')
            self.current_model_name = model_name.upper()
            return True
        except Exception as e:
            print(f"❌ Error loading {model_name}: {e}")
            return False

    def get_audio_prediction(self, audio_path):
        try:
            y, sr = librosa.load(audio_path, sr=22050, duration=3.0)
            rms = np.sqrt(np.mean(y**2))
            if rms < 0.02: return 0.0, "Silence"

            y_trimmed, _ = librosa.effects.trim(y, top_db=20)
            mfcc = librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=self.n_mfcc)
            vector = np.concatenate([np.mean(mfcc, axis=1), np.std(mfcc, axis=1)])
            
            vector_scaled = self.audio_scaler.transform([vector])
            probs = self.audio_model.predict_proba(vector_scaled)[0]
            
            anger_prob = probs[self.audio_angry_idx]
            
            # --- THRESHOLD LOGIC ---
            happy_idx = list(self.audio_le.classes_).index('happy') if 'happy' in self.audio_le.classes_ else -1
            happy_prob = probs[happy_idx] if happy_idx != -1 else 0.0

            if anger_prob > 0.15 and anger_prob > (happy_prob * 0.5):
                return anger_prob, "angry (boosted)"

            max_idx = np.argmax(probs)
            raw_label = self.audio_le.inverse_transform([max_idx])[0]

            if raw_label == 'happy':
                if probs[max_idx] < 0.55: return anger_prob, "other"
                zcr = np.mean(librosa.feature.zero_crossing_rate(y))
                if zcr > 0.25: return 0.0, "static_noise"

            return anger_prob, raw_label
        except Exception as e:
            print(f"Audio Error: {e}")
            return 0.0, "Error"

    def get_video_prediction(self, frame):
        try:
            vector = self.video_extractor.get_features(frame)
            if vector is None: return 0.0, "No Face"
            
            vector_scaled = self.video_scaler.transform([vector])
            probs = self.video_model.predict_proba(vector_scaled)[0]
            
            anger_prob = probs[self.video_angry_idx]
            max_idx = np.argmax(probs)
            label = self.video_le.inverse_transform([max_idx])[0]
            return anger_prob, label
        except Exception as e:
            print(f"⚠️ Video Exception: {e}")
            return 0.0, "Error"

# The last step

In [30]:
import threading
import time
import soundfile as sf
import sounddevice as sd


# --- CONFIG ---
CHUNK_DURATION = 3.0
SAMPLE_RATE = 22050
TEMP_AUDIO_FILE = "live_input.wav"

# --- GLOBALS ---
current_audio_prob = 0.0
current_audio_label = "Listening..."
current_video_prob = 0.0
current_video_label = "Waiting..."
current_main_label = "INITIALIZING"
is_running = True

def audio_listener(fusion_engine):
    global current_audio_prob, current_audio_label, is_running
    while is_running:
        try:
            recording = sd.rec(int(CHUNK_DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1)
            sd.wait()
            sf.write(TEMP_AUDIO_FILE, recording, SAMPLE_RATE)
            
            prob, label = fusion_engine.get_audio_prediction(TEMP_AUDIO_FILE)
            current_audio_prob = prob
            current_audio_label = label
            # print(f"🎤 Audio: {label} ({prob:.2f})")
        except Exception as e:
            print(f"Audio Error: {e}")
            time.sleep(1)

def draw_text_with_box(img, text, pos, bg_color=(0,0,0), text_color=(255,255,255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.6
    thickness = 2
    (t_w, t_h), _ = cv2.getTextSize(text, font, scale, thickness)
    x, y = pos
    cv2.rectangle(img, (x - 5, y - t_h - 5), (x + t_w + 5, y + 5), bg_color, -1)
    cv2.putText(img, text, (x, y), font, scale, text_color, thickness)
    return t_w

def main():
    global current_video_prob, current_video_label, current_main_label, is_running

    if not os.path.exists(casc_path):
        print(f"❌ ERROR: Haar Cascade not found at: {casc_path}")
        return

    try:
        fusion = AngryMeter()
    except Exception as e:
        print(f"❌ Model Load Error: {e}")
        return

    # Audio Thread
    t = threading.Thread(target=audio_listener, args=(fusion,))
    t.daemon = True
    t.start()
    
    cap = cv2.VideoCapture(0)
    draw_cascade = cv2.CascadeClassifier(casc_path)
    
    frame_count = 0
    final_score = 0.0
    
    print("\n📷 LIVE. Controls:")
    print("   [1] SVM  [2] Random Forest  [3] KNN")
    print("   [4] Gradient Boosting  [5] Logistic Regression  [q] Quit")
    
    while True:
        ret, frame = cap.read()
        if not ret: break
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = draw_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        if frame_count % 5 == 0:
            prob, label = fusion.get_video_prediction(frame)
            current_video_prob = prob
            current_video_label = label
            #final_score = fusion.predict_multimodal(current_audio_prob, current_video_prob)
            
            if current_video_prob > 0.6: current_main_label = "DANGER: ANGRY"
            elif current_video_prob < 0.4: current_main_label = "SAFE"
            else: current_main_label = "CAUTION"

        # --- UI LAYOUT ---
        h, w, _ = frame.shape
        
        # 1. Model Indicator (Top Left)
        model_text = f"MODEL: {fusion.current_model_name}"
        draw_text_with_box(frame, model_text, (20, 30), bg_color=(100, 0, 100)) # Purple

        # 2. Labels
        vid_text = f"VIDEO: {current_video_label.upper()}"
        draw_text_with_box(frame, vid_text, (20, 70), bg_color=(0,0,0))
        
        aud_text = f"AUDIO: {current_audio_label.upper()}"
        (aud_w, _), _ = cv2.getTextSize(aud_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        draw_text_with_box(frame, aud_text, (w - 20 - aud_w, 70), bg_color=(0,0,0))

        # 3. Bar
        bar_y1 = 90
        bar_y2 = 110 
        cv2.rectangle(frame, (50, bar_y1), (w-50, bar_y2), (40, 40, 40), -1)
        
        if current_video_prob < 0.4: bar_color = (0, 255, 0)
        elif current_video_prob < 0.7: bar_color = (0, 255, 255)
        else: bar_color = (0, 0, 255)

        fill_width = int((w-100) * current_video_prob)
        cv2.rectangle(frame, (50, bar_y1), (50 + fill_width, bar_y2), bar_color, -1)
        cv2.rectangle(frame, (50, bar_y1), (w-50, bar_y2), (255, 255, 255), 2)

        # 4. Status
        (stat_w, _), _ = cv2.getTextSize(current_main_label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
        center_x = int((w / 2) - (stat_w / 2))
        draw_text_with_box(frame, current_main_label, (center_x, 150), bg_color=bar_color, text_color=(0,0,0))

        cv2.imshow('Manager Angry Meter', frame)
        frame_count += 1
        
        # --- CONTROLS ---
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        elif key == ord('1'): fusion.load_models("svm")
        elif key == ord('2'): fusion.load_models("rf")
        elif key == ord('3'): fusion.load_models("knn")
        elif key == ord('4'): fusion.load_models("gb")
        elif key == ord('5'): fusion.load_models("lr")

    is_running = False
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

🔄 Initializing Fusion Engine...
🔄 Switching to SVM...
✅ System Ready.

📷 LIVE. Controls:
   [1] SVM  [2] Random Forest  [3] KNN
   [4] Gradient Boosting  [5] Logistic Regression  [q] Quit
🔄 Switching to RF...
🔄 Switching to KNN...
🔄 Switching to GB...
🔄 Switching to LR...
🔄 Switching to SVM...
🔄 Switching to RF...
🔄 Switching to KNN...
🔄 Switching to SVM...
🔄 Switching to KNN...
🔄 Switching to GB...
🔄 Switching to KNN...
🔄 Switching to RF...
🔄 Switching to SVM...
🔄 Switching to LR...
